In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.initializers import he_normal
%matplotlib inline

# Display only top 5 and bottom 5 rows
pd.set_option('display.max_rows', 10)

Using TensorFlow backend.


In [2]:
# Set random seed for reproducibility
# Note: CuDNN is usually non-deterministic
# (can't determine which of the ~3000 threads finish earlier)
# And floating points reduction is not perfectly associative due to ULP rounding
import numpy as np
np.random.seed(1337)
import tensorflow as tf
tf.set_random_seed(1337)

In [3]:
df_train = pd.read_csv('./data/train_aWnotuB.csv', parse_dates=[0], infer_datetime_format=True)
df_train

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
48115,2017-06-30 19:00:00,4,11,20170630194
48116,2017-06-30 20:00:00,4,30,20170630204
48117,2017-06-30 21:00:00,4,16,20170630214
48118,2017-06-30 22:00:00,4,22,20170630224


# Preprocessing

In [4]:
train = df_train.pivot(index='DateTime',columns='Junction', values='Vehicles')
train

Junction,1,2,3,4
DateTime,,,,
2015-11-01 00:00:00,15.0,6.0,9.0,NaN
2015-11-01 01:00:00,13.0,6.0,7.0,NaN
2015-11-01 02:00:00,10.0,5.0,5.0,NaN
2015-11-01 03:00:00,7.0,6.0,1.0,NaN
2015-11-01 04:00:00,9.0,7.0,2.0,NaN
...,...,...,...,...
2017-06-30 19:00:00,105.0,34.0,33.0,11.0
2017-06-30 20:00:00,96.0,35.0,31.0,30.0
2017-06-30 21:00:00,90.0,31.0,28.0,16.0


## Remove Nan (0 vehicle)

In [5]:
train = train.fillna(0)

## Normalize the data on 0, 1

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))

In [8]:
train[train.columns] = scaler.fit_transform(train[train.columns])

In [9]:
train

Junction,1,2,3,4
DateTime,,,,
2015-11-01 00:00:00,0.066225,0.106383,0.044693,0.000000
2015-11-01 01:00:00,0.052980,0.106383,0.033520,0.000000
2015-11-01 02:00:00,0.033113,0.085106,0.022346,0.000000
2015-11-01 03:00:00,0.013245,0.106383,0.000000,0.000000
2015-11-01 04:00:00,0.026490,0.127660,0.005587,0.000000
...,...,...,...,...
2017-06-30 19:00:00,0.662252,0.702128,0.178771,0.305556
2017-06-30 20:00:00,0.602649,0.723404,0.167598,0.833333
2017-06-30 21:00:00,0.562914,0.638298,0.150838,0.444444


# Generate rolling forecast features

In [10]:
nb_forecast_per_junction = 24 * (31 + 31 + 30 + 31) # Days in jul + aug + sep + oct

In [11]:
nb_forecast_per_junction

2952

In [12]:
nb_forecast_per_junction * 4

11808

That will certainly not fit in the GPU VRAM --> We will get inspiration from seq2seq models and do a sliding window of time that matches a week.

In [13]:
num_feats = 4
seq_len = 24 * 7 * 2 # Sequence of one week of traffic: 168
num_outputs = 4
num_hidden = 80 # Keep 80 * 2 weeks of hidden state
bs = 768
epochs = 50

We originally had 14592 rows, we will generate sequences of predictions from it.

In [14]:
def make_input_seqs(data, seq_len, train_split=0.9):
    seq_len = seq_len + 1
    result = []
    for index in range(len(data) - seq_len):
        result.append(data[index: index + seq_len, :])
    result = np.array(result) # shape (14423, 168, 4)
    train_ind = round(train_split * result.shape[0])
    train = result[:int(train_ind), :, :]
    x_train = train[:, :-1, :]
    y_train = train[:, -1, :]
    x_test = result[int(train_ind):, :-1, :]
    y_test = result[int(train_ind):, -1, :]

    return [x_train, y_train, x_test, y_test]

In [15]:
X_train, y_train, X_test, y_test = make_input_seqs(train.values, seq_len)

# Generate model

## Loss function

In [16]:
import keras.backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

## Neural net

In [17]:
def Net(num_feats, seq_len, num_hidden, num_outputs):
    model = Sequential()

    # Encoder RNN
    model.add(LSTM(
        input_shape=(seq_len,num_feats),
        activation = 'relu', # default is tanh
        kernel_initializer = he_normal(seed=1337),
        return_sequences=True, units=seq_len))
    model.add(Dropout(0.2))
    
    # Deoder RNN
    model.add(LSTM(
        num_hidden,
        activation = 'relu', # default is tanh
        kernel_initializer = he_normal(seed=1337),
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(num_outputs))
    model.add(Activation("linear"))

    model.compile(loss= root_mean_squared_error, optimizer="adam")
    return model

In [18]:
model = Net(num_feats, seq_len, num_hidden, num_outputs)

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 336, 336)          458304    
_________________________________________________________________
dropout_1 (Dropout)          (None, 336, 336)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 80)                133440    
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 324       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 592,068
Trainable params: 592,068
Non-trainable params: 0
_________________________________________________________________


### Callbacks

In [20]:
from keras.callbacks import History, ModelCheckpoint, CSVLogger, EarlyStopping
import time

In [ ]:
history = History()
checkpointer = ModelCheckpoint(filepath="checkpoints/" + time.strftime("%Y-%m-%d_%H%M-")+"seq2seq-relu-normalized.hdf5",
                               verbose=1, save_best_only=False)
csv_logger = CSVLogger("checkpoints/" + time.strftime("%Y-%m-%d_%H%M-")+'training.log')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

# Training

In [ ]:
model.fit(X_train, y_train,
          batch_size=bs,
          epochs=epochs,
          validation_split=0.05,
          callbacks=[history,checkpointer,csv_logger,early_stop])

Train on 12188 samples, validate on 642 samples
Epoch 1/50

 6912/12188 [================>.............] - ETA: 11s - loss: 0.1058

# Validation

## Validation by feeding truth values

In [ ]:
def plot_preds(y_truth, y_pred):
    for junction in range(4):
        plt.figure
        plt.plot(y_truth[:,junction], color = 'green', label = 'Real traffic')
        plt.plot(y_pred[:,junction], color = 'red', label = 'Predicted traffic')
        plt.title('Traffic Forecasting at junction %i' % (junction+1))
        plt.xlabel('Number of hours from Start')
        plt.ylabel('Traffic')
        plt.legend()
        plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plot_preds(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
rmse(y_test, y_pred)

## Validation by feeding predicted values

In [ ]:
def pred_seq(model, to_pred, w_size, num_preds=None):
    if num_preds == None:
        num_preds = to_pred.shape[0]
    current = to_pred[0]
    predicted = []
    for i in range(num_preds):
        predicted.append(np.round(model.predict(current[np.newaxis,:,:])[0,:]))
        current = current[1:]
        current = np.insert(current, [w_size-1], predicted[-1], axis=0)
    return np.asarray(predicted)

In [ ]:
seqpreds = pred_seq(model, X_test, seq_len)

In [ ]:
seqpreds.shape

In [ ]:
plot_preds(y_test, seqpreds)

In [ ]:
rmse(y_test, seqpreds)